<a href="https://colab.research.google.com/github/taunoe/jupyter-notebooks/blob/main/Sagedad_s%C3%B5nad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sagedad sõnad

*Praktilise näite autor: Mark Fišel*

Selles praktilises näites vaatame, kui suurt osakaalu tekstist moodustavad selle kõige sagedamad sõnad. Näiteks vastame küsimusele, kui suurest osast keelest (tekstist) me aru saame, kui teme ainult 100 sagedaimat sõna. Arvestame sõnade algvorme, ning nende leidmiseks kasutame EstNLTK-d.

Kõigepealt laeme alla tekstikorpuse, kus on 2015. aasta uudisartiklid eesti ajalehtedest.

In [1]:
!wget http://data.statmt.org/wmt18/translation-task/news.2015.et.shuffled.deduped.gz
!gunzip news.2015.et.shuffled.deduped.gz
!wc -lw news.2015.et.shuffled.deduped

--2021-04-14 05:39:09--  http://data.statmt.org/wmt18/translation-task/news.2015.et.shuffled.deduped.gz
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44528040 (42M) [application/x-gzip]
Saving to: ‘news.2015.et.shuffled.deduped.gz’

news.2015.et.shuffl 100%[===================>]  42.46M  5.76MB/s    in 8.0s    

2021-04-14 05:39:18 (5.30 MB/s) - ‘news.2015.et.shuffled.deduped.gz’ saved [44528040/44528040]

   962663  13002539 news.2015.et.shuffled.deduped


Nagu näha, on korpuses kokku 962 tuhat lauset ja 13 miljonit sõna.

Edasi installime EstNLTK.

In [3]:
!pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple  estnltk==1.6.7beta

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple
     |████████████████████████████████| 99.4MB 59kB/s 
     |████████████████████████████████| 747kB 5.6MB/s 


Järgmisena otsustame, mitu sagedaimat sõna meid huvitab. Proovige muuta muutuja "top_sõnade_arv" väärtust 100 asemel 200ks või 500ks.

Samuti piirame me lausete arvu töötlemiseks, kuna algvormide analüüs võtab aega. Näiteks, 3000 lause töötlemine võtab umbes 2 minutit aega. Kui viitsite kauem oodata, võite töödelda rohkem lauseid:

In [4]:
top_sõnade_arv = 100
mitu_lauset_lugeda = 3000

Nüüd loeme teksti sisse:

In [5]:
laused = []

with open('news.2015.et.shuffled.deduped', 'r') as fh:
  while len(laused) < mitu_lauset_lugeda:
    laused.append(fh.readline())

toortekst = " ".join(laused)

print("Esimene lause: ", laused[0])
print("Viimane lause: ", laused[-1])

Esimene lause:  Flush annab hädalisele teada, millised WCd tema läheduses leiduvad, kas need maksulised või sobilikud näiteks ratastoolis istujale.

Viimane lause:  Lõppude lõpuks on käimas ju ettevalmistused Oscarite galaks - seda tahavad kindlasti paljud näha!



Järgmisena koostame kõikide sõnade nimekirja.

Selles kohas on teie ülesanne muuta koodi nii, et sõnavormide asemel arvestaks programm algvorme. Selleks asendage järgmise koodijupi esimesel koodireal "kasuta_algvorme" muutuja väärtust uue väärtusega "True", ning käivitage kood uuesti.

**Selle koodijupi käivitamine võtab vähemalt paar minutit aega.**

In [7]:
kasuta_algvorme = True

import re
from estnltk import Text

analüüsitud_tekst = Text(toortekst)
_ = analüüsitud_tekst.tag_layer()

teksti_sõnad = []

for lause in analüüsitud_tekst.sentences:
  teksti_sõnad += [(sõna.morph_analysis.lemma[0] if kasuta_algvorme else sõna.text) for sõna in lause if re.search(r'[a-zäöüõšž]', sõna.text)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Nüüd koostame N sagedaima sõna (algvormi) nimekirja.

In [8]:
from nltk import FreqDist
sagedused = FreqDist(teksti_sõnad)

n_sagedaimat_sõna = {sõna for sõna,_ in sagedused.most_common(top_sõnade_arv)}

print("10 sagedaimat sõnet:")
for sõna, sagedus in sagedused.most_common(10):
  print("{0} ({1})".format(sõna, sagedus))

10 sagedaimat sõnet:
olema (1713)
ja (1045)
see (616)
et (541)
tema (454)
ei (369)
kui (343)
mis (333)
ka (322)
ning (306)


 Lõpuks arvutame N sagedaima sõna (algvormi) osakaalu terves tekstis.

In [9]:
sagedaimate_sõnade_arv = sum([1 for sõna in teksti_sõnad if sõna in n_sagedaimat_sõna])

print("Lauseid: {0}".format(len(laused)))
print("Sõnu: {0}".format(len(teksti_sõnad)))
print("{0} sagedaima sõna osakaal tekstis: {1:0.4}%".format(top_sõnade_arv, 100 * sagedaimate_sõnade_arv / len(teksti_sõnad)))

Lauseid: 3000
Sõnu: 39091
100 sagedaima sõna osakaal tekstis: 34.08%
